# Crawler Web pages

In [11]:
# load dependency libraries
import requests
import os
import re
import pickle
from bs4 import BeautifulSoup
from collections import deque

In [60]:
domain = "uic.edu"
start_url = "https://www.cs.uic.edu/"                ######## update url

pages_folder = "./FetchedPages/"

pickle_folder = "./PickleFiles/"
os.makedirs(pickle_folder, exist_ok=True)

# file extensions to ignore while crawling pages
ignore_ext = [
    '.pdf', '.doc', '.docx', '.ppt', '.pptx', '.xls', '.xlsx', '.css', '.js',
    '.aspx', '.png', '.jpg', '.jpeg', '.JPG', '.gif', '.svg', '.ico', '.mp4',
    '.avi', '.tar', '.gz', '.tgz', '.zip'
]

crawl_limit = 3000

# to make sure error log file is initially empty
error_file = "error_log.txt"
f = open(error_file, "w+")
# f.truncate()
f.close()

# queue to perform BFS web traversal
url_q = deque()
url_q.append(start_url)

# list to keep track of traversed URLs
urls_crawled = []
urls_crawled.append(start_url)

# dict to track pages fetched and stored in folder
pages_crawled = {}
page_no = 0

while url_q:
    
    try:
        url = url_q.popleft()               # fetch the first URL from the queue
        rqst = requests.get(url)            # get html code of web page

        if (rqst.status_code == 200):
            
            soup = BeautifulSoup(rqst.text, 'lxml')
            tags_extracted = soup.find_all('a')                 # extract all 'a' tags from page
            
            if len(tags_extracted) != 0:                        # to reject pages which don't link to another page
                pages_crawled[page_no] = url    

                output_file = pages_folder + str(page_no)

                os.makedirs(os.path.dirname(output_file), exist_ok=True)     # create file to store html code

                with open(output_file, "w", encoding="utf-8") as file:
                    file.write(rqst.text)
                file.close()

                for tag in tags_extracted:

                    l = tag.get('href')

                    if l is not None and l.startswith("http"):

                        if not any(ext in l for ext in ignore_ext):

                            if l not in urls_crawled and domain in l:
    #                             print(l)
                                url_q.append(l)                 # valid URL to append to the queue
                                urls_crawled.append(l)

                if (len(pages_crawled) > crawl_limit):
                    break                                       # stop crawling when reached limit

                page_no += 1
            
#             else:
#                 print(url,soup.getText())

    except Exception as e:
        with open(error_file, "a+") as log:                  # add error message to error log
            log.write(f"Could not connect to {url}")
            log.write(f"\nError occured: {e}\n\n")
        file.close()

        print("Could not connect to ", url)
        print("Error occured: ", e, " \n")
        continue
        

# Pickling the dict of crawled pages
with open(pickle_folder + 'pages_crawled.pickle', 'wb') as f:
    pickle.dump(pages_crawled,f)

Could not connect to  http://www.uic.edu/orgs/ccspd/
Error occured:  HTTPConnectionPool(host='ccspd.org.uic.edu', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002AFFE549160>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))  

http://cas.uic.edu/it/ 



index





Could not connect to  https://vcsaonline.uic.edu/lms
Error occured:  HTTPSConnectionPool(host='vcsaonline.uic.edu', port=443): Max retries exceeded with url: /lms (Caused by SSLError(SSLCertVerificationError("hostname 'vcsaonline.uic.edu' doesn't match 'dev.sa.uic.edu'")))  

http://commuter.uic.edu/ 



Commuter Students Resource Center





Could not connect to  https://www.ready.uic.edu
Error occured:  HTTPSConnectionPool(host='www.ready.uic.edu', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError("hostname 'www.ready.uic.edu' doesn't match '*.accc.uic.edu'")))  

https://go.uic.edu/

http://ask.library.uic.edu/widget_standalone.php?la_widget_id=755 






Widget





http://go.uic.edu/linkedinlearning LinkedIn Authentication RequestLinkedIn Authentication RequestRedirecting...You are automatically being redirected to the authentication service. If the browser appears to be hung up after 15-20 seconds, try reloading the page.Note: Since your browser does not support JavaScript, you must press the
Continue button once to proceed to the authentication service.fs.config({"failureRedirect":"http://www.linkedin.com/","uniEscape":true,"xhrHeaders":{"X-FS-Origin-Request":"/checkpoint/enterprise/login/43607124?application=learning&appInstanceId=47820636&authModeId=6559184309981450240&authModeName=iTrust-Proxy","X-FS-Page-Id":"checkpointEnterpriseLogin"}});
http://connect.uic.edu 











 - UIC Connection

	
    var appInsights=window.appInsights||function(a){
        function b(a){c[a]=function(){var b=arguments;c.queue.push(function(){c[a].apply(c,b)})}}var c={config:a

http://l.facebook.com/l.php?u=http%3A%2F%2Fwww.uic.edu%2F&h=AT3RaayuQ7dtag40Gji8q_df4Qnpq3ns0lye2BPd9n03MhmPKWJTbxMI5ArXUUId3byeGD0zBtQioTCqjWiX-ghLIb0qEIH736s0ji_95etNVn3RJxMGr_gUjh_O1vuetVUNTQ5qpUqVfYKlDLro5ZWr3EwsUP0e document.location.replace("http:\/\/www.uic.edu\/");
http://l.facebook.com/l.php?u=http%3A%2F%2Fwww.uic.edu%2F&h=AT3A71AGewsdMBSmHpBz4XdvtraK0NPPLjZ2cjDVab2EbxGxsdlFtCZQDMIaglhpDdBOxqH3HPXaQunUMrYilzsTLekq_LFf0W8GHJA3ynCOJE_3ERoKHP6eaVPkF5FK8ZDZI-AN90R_0-QlG1bcaarxFbdxq69SaQE document.location.replace("http:\/\/www.uic.edu\/");
https://apps.honors.uic.edu/Fellows/current-fellows.php 

Fellows

table{width:90%;margin:0 auto;}
thead tr,th{text-align:left;background-color:silver;}



 First NameLast NameDepartmentCollegeAddressMail CodeEmail Address
1CrystalTseACE2982 SSB327tsecryst@uic.edu
2LeilaAmiriAdmissions & Recruitment 165 CMET783lamiri@uic.edu
3MarkMartellAsian American Resource and Cultural Center101 TH203mmartell@uic.edu
4NickArdingerCampus Housing335 SRH579ardi

Could not connect to  http://www.psych.uic.edu/
Error occured:  HTTPSConnectionPool(host='www.psych.uic.edu', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))  

Could not connect to  http://www.psych.uic.edu/mhsrp/
Error occured:  HTTPSConnectionPool(host='www.psych.uic.edu', port=443): Max retries exceeded with url: /mhsrp/ (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))  

Could not connect to  http://www.psych.uic.edu/MHSRP/
Error occured:  HTTPSConnectionPool(host='www.psych.uic.edu', port=443): Max retries exceeded with url: /MHSRP/ (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))  

Could not connect to  http://www.psych.uic.edu/eidp/
Error occured:  HTTPSConnectionPool(ho

https://today.uic.edu/events/feed 

UIC Today Events
https://today.uic.edu/events
		UIC Events
http://blogs.law.harvard.edu/tech/rss
Wed, 29 Apr 2020 19:20:35 +0000


Earth Day 2020: Plastic-Free UIC!
https://today.uic.edu/events/earth-day-2020-plastic-free-uic
					https://today.uic.edu/events/earth-day-2020-plastic-free-uic
Thu, 09 Apr 2020 05:00:00 +0000
04/09/2020 - 04/30/2020UIC's first virtual Earth Day ]]>


Competitive Gaming Club Discord
https://today.uic.edu/events/competitive-gaming-club-discord-2020-04-30
					https://today.uic.edu/events/competitive-gaming-club-discord-2020-04-30
Thu, 30 Apr 2020 05:00:00 +0000
04/30/2020Gaming events scheduled]]>


UIC Campus Recreation free online workouts
https://today.uic.edu/events/uic-campus-recreation-free-online-workouts-2020-04-30
					https://today.uic.edu/events/uic-campus-recreation-free-online-workouts-2020-04-30
Thu, 30 Apr 2020 05:00:00 +0000
04/30/2020Virtual events sponsored by Campus Recreation]]>


Black Hair Quilt Proje

http://researchguides.uic.edu/go.php?c=10002071 Shibboleth Authentication Request

If your browser does not continue automatically, click 

<!--
function EZproxyCheckBack() {  var goForward = (document.EZproxyTrack.back.value==1);  document.EZproxyTrack.back.value=2;  document.EZproxyTrack.back.defaultValue=2;  if (goForward) { document.EZproxyForm.submit(); }}
-->


http://ask.library.uic.edu/widget_standalone.php?la_widget_id=6169 






Widget





Could not connect to  https://today.uic.edu/teaching-learning-and-working-resources?utm_source=mailpoet&utm_medium=email&utm_campaign=message-from-the-chancellor_551
Error occured:  HTTPSConnectionPool(host='today.uic.edu', port=443): Max retries exceeded with url: /teaching-learning-and-working-resources?utm_source=mailpoet&utm_medium=email&utm_campaign=message-from-the-chancellor_551 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002AFFE928880>: Failed to establish a new connection: [WinError 1

In [61]:
len(pages_crawled)

3001

In [62]:
# pickle_folder = "./PickleFiles/"
# os.makedirs(pickle_folder, exist_ok=True)

In [64]:
with open(pickle_folder + 'pages_crawled.pickle', 'rb') as f:
    pages = pickle.load(f)

In [65]:
# pages == pages_crawled

In [75]:
# pages_crawled[203]